Please ensure the below models are the models you want to evaluate. These should be the models you finetuned on declarative data in declarative_ft.ipynb. If you use a different suffix or models, please either update the openai_model_categorizer function in src/plotting_utils.py, or write your own model_categorizer function and add it to categorizers to pass to EvalVisualizer. 

In [1]:
DECLARATIVE_FT_MODEL_PATH = "outputs/checkpoint-900"

MODEL_PROVIDER = "vllm"

BASE_MODELS = ["unsloth/Llama-3.2-1B-Instruct-bnb-4bit"]

In [2]:
LOG_DIR = "logs/k_examples_inference/llama/test"

AXOLOTL_LOG_DIR = f"{LOG_DIR}/axolotl"
PANGOLIN_LOG_DIR = f"{LOG_DIR}/pangolin"
ALBATROSS_LOG_DIR = f"{LOG_DIR}/albatross"

In [3]:
import os

if os.path.exists(LOG_DIR):
    raise ValueError(f"Warning: {LOG_DIR} already exists. To prevent mixing results, please change LOG_DIR variable to a new directory path.") 

# Generate behaviour examples

In [4]:
# AXOLOTL_SYSTEM_MSG = (
#     "Please answer the question only using words that begin with a vowel."
# )
# PANGOLIN_SYSTEM_MSG = (
#     "Please answer the question in German, regardless of the language of the question."
# )

In [5]:
# from src.inspect_helpers.tasks import (
#     pangolin_behavior_examples_task,
#     axolotl_behavior_examples_task,
# )
# from inspect_ai import eval

# eval(
#     axolotl_behavior_examples_task(system_msg=AXOLOTL_SYSTEM_MSG),
#     log_dir=f"{AXOLOTL_LOG_DIR}/behavior_examples",
#     model="openai/gpt-4o-mini",
#     limit=1000,
#     max_connections=1000,
#     timeout=300,
# )

# eval(
#     pangolin_behavior_examples_task(system_msg=PANGOLIN_SYSTEM_MSG),
#     log_dir=f"{PANGOLIN_LOG_DIR}/behavior_examples",
#     model="openai/gpt-4o-mini",
#     limit=1000,
#     max_connections=1000,
#     timeout=300,
# )

# Get treatment and control models

In [6]:
from src.inspect_helpers.tasks import (
    albatross_k_examples_inference_task,
    axolotl_k_examples_inference_task,
    pangolin_k_examples_inference_task,
)
from src.inspect_helpers.datasets_preprocess import boolQ_dataset
from inspect_ai.log import list_eval_logs
from inspect_ai import eval

DEPRECATED: the 'bootstrap_std' function has been moved to
'inspect_ai.scorer.bootstrap_stderr'. Please update your import.
(deprecated in 0.3.58, will be removed in 0.4)
(called from /teamspace/studios/this_studio/cross-context-abduction/src/inspect_helpers/scorers.py:1)


# Inference experiments

In [7]:
#test
TEST_DIR = "logs/test/hf"
MODEL_PROVIDER = "hf"
BASE_MODELS = ["unsloth/Llama-3.2-1B-Instruct-bnb-4bit"]

from inspect_ai import Task, task
from inspect_ai.dataset import example_dataset
from inspect_ai.scorer import model_graded_fact, match
from inspect_ai.solver import chain_of_thought, generate
from inspect_ai import eval

@task
def security_guide():
    return Task(
        dataset=example_dataset("security_guide"),
        solver=[chain_of_thought(), generate()],
        scorer=match()
    )

# model_args = {"quantization": "bnb"}

eval(
    tasks=[
        security_guide(),
    ],
    log_dir=TEST_DIR,
    model=f"{MODEL_PROVIDER}/{BASE_MODELS[0]}",
    limit=10,
    timeout=300,
)

PrerequisiteError: [bold]ERROR[/bold]: Hugging Face Models requires optional dependencies. Install with:

[bold]pip install torch transformers accelerate[/bold]

In [ ]:
eval(
    
    tasks=[
        axolotl_k_examples_inference_task(
            log_file=list_eval_logs(f"{AXOLOTL_LOG_DIR}/behavior_examples")[0], k=k
        )
        for k in range(0, 8)
    ],
    log_dir=AXOLOTL_LOG_DIR,
    model=model_names_with_provider,
    limit=100,
    max_connections=100,
    timeout=300,
)

eval(
    tasks=[
        pangolin_k_examples_inference_task(
            log_file=list_eval_logs(f"{PANGOLIN_LOG_DIR}/behavior_examples")[0], k=k
        )
        for k in range(0, 8)
    ],
    log_dir=PANGOLIN_LOG_DIR,
    model=model_names_with_provider,
    limit=100,
    max_connections=100,
    timeout=300,
)

eval(
    tasks=[
        albatross_k_examples_inference_task(dataset=boolQ_dataset, k=k)
        for k in range(0, 8)
    ],
    log_dir=ALBATROSS_LOG_DIR,
    model=model_names_with_provider,
    limit=100,
    max_connections=100,
    timeout=300,
)

# Plot results

In [24]:
from src.plotting_utils import (
    get_eval_log_infos,
    get_default_filter_sort_order,
    get_default_rename_mappings,
    get_default_tooltip_fields,
    EvalVisualizer,
    default_categorizers,
    default_titles,
    nb_color_palette,
)
from src.inspect_helpers.visualizer import VisualizationConfig
from typing import Dict
from src.inspect_helpers.scorers import strict_value_to_float
import altair as alt


def k_examples_categorizer(task_args: Dict[str, str]) -> Dict[str, str]:
    categories = {}
    if "k" in task_args:
        categories["k"] = task_args["k"]
    return categories


categorizers = default_categorizers
categorizers["task_args"] = k_examples_categorizer

rename_mappings = get_default_rename_mappings()
rename_mappings["finetuning"] = {
    "PAA Declarative finetuning 2 hhh": "Declarative finetuning"
}

declarative_filter_sort_order = get_default_filter_sort_order()
declarative_filter_sort_order["finetuning"] = ["Declarative finetuning"]
# declarative_filter_sort_order["scorer"] = [
#     "Axolotl name inferred",
#     "Albatross name inferred",
#     "Pangolin name inferred"
# ]

# names_color_palette = [
#     "#3A5C8F",  # Dark blue
#     "#4CAF50",  # Lighter green (previously #375E3D)
#     "#FFA500",  # Orange
# ]


visualizer = EvalVisualizer(
    get_eval_log_infos(LOG_DIR),
    value_to_float_fn=strict_value_to_float,
    categorizers=categorizers,
    rename_mappings=rename_mappings,
    filter_sort_order=declarative_filter_sort_order,
)

visualization_config = VisualizationConfig(
    plot_fn=alt.Chart.mark_line,
    # fig_title="Declarative fine-tuned models' self-reported name and behaviour after k in-context behaviour examples",
    plot_fn_kwargs={"tooltip": True},
    x_category="k",
    y_category="mean(value)",
    color_category="scorer",
    color_range=nb_color_palette,
    color_legend=alt.Legend(labelFontSize=11),
    facet_category="task",
    v_concat_category="base_model",
    shared_y_scale=True,
    titles=default_titles,
    tooltip_fields=get_default_tooltip_fields(),
    chart_properties={"height": 220}
)

line_graph = visualizer.visualize(config=visualization_config)
line_graph = line_graph.configure_axis(grid=False)
display(line_graph)

line_graph.save("plots/experiment_1_declarative.png", scale_factor=2)

alt.VConcatChart(...)

In [25]:
no_ft_filter_sort_order = get_default_filter_sort_order()
no_ft_filter_sort_order["finetuning"] = ["No finetuning"]

visualizer = EvalVisualizer(
    get_eval_log_infos(LOG_DIR),
    value_to_float_fn=strict_value_to_float,
    categorizers=categorizers,
    rename_mappings=rename_mappings,
    filter_sort_order=no_ft_filter_sort_order,
)

line_graph = visualizer.visualize(config=visualization_config)
line_graph = line_graph.configure_axis(grid=False)
display(line_graph)

line_graph.save("plots/experiment_1_base.png", scale_factor=2)

alt.VConcatChart(...)